In [5]:
include("../LogLoss/RealRealHighDimension.jl")
include("./ForecastingMainNew.jl");
using JLD2

Load in the data containing the following:
- X_train_scaled
- y_train
- X_test_scaled
- y_test
- mps
- opts

In [15]:
dloc = "/Users/joshua/Desktop/QTNML_paper/QuantumInspiredML/Interpolation/paper/italypower/mps_saves/ipd_legendre_chi60_d12_twoClass.jld2";
f = jldopen(dloc, "r");

JLDFile /Users/joshua/Desktop/QTNML_paper/QuantumInspiredML/Interpolation/paper/italypower/mps_saves/ipd_legendre_chi60_d12_twoClass.jld2 (read-only)
 ├─🔢 X_train_scaled
 ├─🔢 y_train
 ├─🔢 X_test_scaled
 ├─🔢 y_test
 ├─🔢 mps
 └─🔢 opts

In [ ]:
opts=Options(; nsweeps=12, chi_max=60, update_iters=1, verbosity=verbosity, dtype=dtype, loss_grad=loss_grad_KLD,
bbopt=BBOpt("CustomGD"), track_cost=track_cost, eta=0.0025, rescale = (false, true), d=12, aux_basis_dim=2, encoding=encoding, 
encode_classes_separately=encode_classes_separately, train_classes_separately=train_classes_separately)
